In [1]:
import numpy as np
import pandas as pd
import datetime
import time

Importing the amazon data

In [2]:
meta = pd.read_csv("NLP_metadata.csv", low_memory=False)
rating = pd.read_csv("NLP_ratings_all_top1000.csv", low_memory=False)

printing dimensions

In [3]:
print("\nmetadata dimensions")
print(meta.shape)
print("\nrating dimensions")
print(rating.shape)


metadata dimensions
(279899, 9)

rating dimensions
(203500, 510)


In [4]:
# Restricting title sizes
def auto_truncate_description(val):
    return val[:1024]
def auto_truncate_title(val):
    return val[:255]

In [5]:
rating.head(3)

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall_x,reviewTime,summary,cleanedreviews,...,worth,wow,write,writing,written,wrong,year,years,yes,young
0,A2EFCYXHNK06IS,5555991584,"Abigail Perkins ""Abby &#34;Reads Too Much&#34...","[4, 5]",978480000,The anthemic title track begins &quot;The Memo...,5.0,"01 3, 2001",Enya Experiments And Succeeds,anthemic title track begins memory trees fourt...,...,0,0,0,0,0,0,0,0,0,0
1,A1WR23ER5HMAA9,5555991584,AKB,"[1, 1]",953424000,Just when I thought Enya couldn't possibly get...,5.0,"03 19, 2000",How to improve upon perfection.,thought enya possibly get better found memory ...,...,0,0,0,0,0,0,0,0,0,0
2,A2IR4Q0GPAFJKW,5555991584,Alexander,"[0, 0]",1393545600,Nice CD for easy listening. My husband and I ...,4.0,"02 28, 2014",Good listening.,nice cd easy listening husband like quite bit ...,...,0,0,0,0,0,0,0,0,0,0


Only using (user-item-rating-timestamp) for now

In [6]:
rating = rating.drop(["reviewerName", "helpful", "reviewText", "summary", "cleanedreviews", "reviewTime"], 1)

In [7]:
# df_ratings = rating[["reviewerID", "asin", "overall", "unixReviewTime"]]
df_ratings = rating
df_ratings.unixReviewTime = pd.to_datetime(df_ratings["unixReviewTime"],unit = 's')
df_ratings["overall_x"] = df_ratings["overall_x"].astype(int)

#changing column names
df_ratings = df_ratings.rename(columns={'reviewerID': 'user_id', 'asin': 'item_id', 'overall_x': 'rating', 'unixReviewTime': 'timestamp'})


In [8]:
df_ratings.head()

,user_id,item_id,timestamp,rating,able,absolutely,acoustic,actually,add,admit,...,worth,wow,write,writing,written,wrong,year,years,yes,young
0,A2EFCYXHNK06IS,5555991584,2001-01-03,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A1WR23ER5HMAA9,5555991584,2000-03-19,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A2IR4Q0GPAFJKW,5555991584,2014-02-28,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A2V0KUVAB9HSYO,5555991584,2000-08-13,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A1J0GL9HCA7ELW,5555991584,2001-12-07,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Parsing the item_ids to an integer format and taking the top 1000 items only

In [9]:
item_filter = pd.DataFrame(df_ratings['item_id'])
item_filter = item_filter.groupby('item_id').size()
item_filter = item_filter.to_frame().reset_index()
item_filter.columns.values[1] = 'count'
item_filter = item_filter.sort_values('count', ascending=False)


In [10]:
print(item_filter['count'].mean())
print(item_filter['count'].max())
print(item_filter['count'].min())
item_filter.head()

203.5
1953
83


,item_id,count
767,B004D1GZ2E,1953
709,B0026P3G12,1926
458,B0000AGWEC,1823
774,B004K4AUZW,1527
575,B000BGR18W,1386


In [11]:
item_filter = item_filter.head(n=1000)

In [12]:
print(item_filter['count'].mean())
print(item_filter['count'].max())
print(item_filter['count'].min())
item_filter.head()

203.5
1953
83


,item_id,count
767,B004D1GZ2E,1953
709,B0026P3G12,1926
458,B0000AGWEC,1823
774,B004K4AUZW,1527
575,B000BGR18W,1386


In [13]:
item_filter = item_filter[['item_id']]

In [14]:
df_ratings.shape

(203500, 504)

In [15]:
df_ratings = pd.merge(left=df_ratings,right=item_filter, left_on='item_id', right_on='item_id')

In [16]:
df_ratings.shape

(203500, 504)

In [17]:
df_ratings = df_ratings.sort_values("user_id")

In [18]:
df_ratings.user_id = df_ratings.user_id.astype("category")

df_ratings["userid_key"] = df_ratings["user_id"].cat.codes
df_ratings["userid_key"] = df_ratings["userid_key"] + 1


In [19]:
df_ratings.head()

,user_id,item_id,timestamp,rating,able,absolutely,acoustic,actually,add,admit,...,wow,write,writing,written,wrong,year,years,yes,young,userid_key
186657,A000710818SILS4SMRH43,B00A3OT5A4,2013-07-18,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
157413,A00167062N853P9T0TEJ5,B004D1GZ2E,2013-02-26,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
16443,A00219482AZDX81KAXHEF,B0000029B0,2013-03-09,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
171594,A00361823YYOF5UOJPIU,B007B6VOII,2012-11-05,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
193049,A00473363TJ8YSZ3YAGG9,B00CBUNUAO,2013-12-07,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


importing items

In [20]:
df_items = meta

In [21]:
df_items.head()

,asin,title,price,imUrl,related,salesRank,categories,description,brand
0,5555991584,Memory of Trees,9.49,http://ecx.images-amazon.com/images/I/51b5WDjd...,"{'also_bought': ['B000002LRT', 'B000002LRR', '...",{'Music': 939190},"[['CDs & Vinyl', 'New Age', 'Celtic New Age'],...",NaN,NaN
1,6308051551,Don't Drink His Blood,8.91,http://ecx.images-amazon.com/images/I/31LT2n7Q...,NaN,NaN,"[['Digital Music', 'Alternative Rock', 'Indie ...",NEW Combo BLUWAVS CD and FLAC FILE,NaN
2,7901622466,On Fire,11.33,http://ecx.images-amazon.com/images/I/21W29WZw...,"{'also_bought': ['B00000282A', 'B0084O8O9S', '...",{'Music': 58799},"[['CDs & Vinyl', 'Christian', 'Rock & Alternat...",NaN,NaN
3,B0000000ZW,Changing Faces,23.64,http://ecx.images-amazon.com/images/I/51H3Bp-3...,"{'also_bought': ['B00000010Z', 'B0000039Q5', '...",{'Music': 68784},"[['CDs & Vinyl', 'Dance & Electronic'], ['CDs ...",NaN,NaN
4,B00000016W,Pet Sounds,9.49,http://ecx.images-amazon.com/images/I/51pDGkXj...,"{'also_bought': ['B0025KVLTM', 'B00007FOMP', '...",{'Music': 77205},"[['CDs & Vinyl', 'Classic Rock'], ['CDs & Viny...",NaN,NaN


In [22]:
# extracting specific columns
df_items = df_items[["asin", "title", "description", "imUrl", "price"]]
df_items[["old_id"]] = df_items[['asin']]
# renaming columns
df_items = df_items.rename(columns={'asin': 'id', 'title': 'title', 'description': 'description', 'imUrl': 'imgurl'})


df_items = pd.merge(left=df_items,right=item_filter, left_on='id', right_on='item_id')


C:\Users\Nabeel\Anaconda2\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [23]:
df_items.head()

,id,title,description,imgurl,price,old_id,item_id
0,5555991584,Memory of Trees,NaN,http://ecx.images-amazon.com/images/I/51b5WDjd...,9.49,5555991584,5555991584
1,B00000016W,Pet Sounds,NaN,http://ecx.images-amazon.com/images/I/51pDGkXj...,9.49,B00000016W,B00000016W
2,B00000016T,Cars,NaN,http://ecx.images-amazon.com/images/I/51pmW%2B...,14.63,B00000016T,B00000016T
3,B0000002ME,Now That I've Found You: Collection,"A poll-winning fiddler since her teens, Alison...",http://ecx.images-amazon.com/images/I/51FYnByO...,8.94,B0000002ME,B0000002ME
4,B00000050T,12-Play,NaN,http://ecx.images-amazon.com/images/I/41tBZM%2...,7.29,B00000050T,B00000050T


In [24]:
# creating keys for items
df_items = df_items.sort_values("id")
df_items['item_key'] = range(len(df_items))

df_items = df_items[["item_key", "title","price", "description", "imgurl","old_id"]]
df_items = df_items.rename(columns={'item_key': 'id'})

df_items = df_items.sort_values("id")
df_items = df_items.iloc[1:]


df_items_key = df_items[["old_id", "id"]]
df_items_key = df_items_key.rename(columns={'id': 'item_key'})

In [25]:
df_items.head()

,id,title,price,description,imgurl,old_id
2,1,Cars,14.63,NaN,http://ecx.images-amazon.com/images/I/51pmW%2B...,B00000016T
1,2,Pet Sounds,9.49,NaN,http://ecx.images-amazon.com/images/I/51pDGkXj...,B00000016W
3,3,Now That I've Found You: Collection,8.94,"A poll-winning fiddler since her teens, Alison...",http://ecx.images-amazon.com/images/I/51FYnByO...,B0000002ME
4,4,12-Play,7.29,NaN,http://ecx.images-amazon.com/images/I/41tBZM%2...,B00000050T
5,5,Backstreet Boys,12.48,The world may be divided into two camps when i...,http://ecx.images-amazon.com/images/I/51N3tp0T...,B00000053X


Now, combining columns with the ratings

In [26]:
df_ratings = pd.merge(df_ratings, df_items_key, left_on = "item_id", right_on="old_id")

In [27]:
df_ratings.head(3)

,user_id,item_id,timestamp,rating,able,absolutely,acoustic,actually,add,admit,...,writing,written,wrong,year,years,yes,young,userid_key,old_id,item_key
0,A000710818SILS4SMRH43,B00A3OT5A4,2013-07-18,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,B00A3OT5A4,903
1,A11RYUB6ITAGVF,B00A3OT5A4,2013-09-07,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2056,B00A3OT5A4,903
2,A13CXQGLSH6QY8,B00A3OT5A4,2013-07-05,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3778,B00A3OT5A4,903


In [28]:

# df_ratings = df_ratings[["userid_key", "item_key", "rating", 'timestamp', "user_id"]]
df_ratings = df_ratings.rename(columns={'user_id': 'old_user_id'})

df_ratings = df_ratings.rename(columns={'userid_key': 'user_id', 'item_key': 'item_id'})

df_ratings = df_ratings.sort_values("user_id")

In [29]:
df_ratings = df_ratings.drop(['old_id'],1)

Final format of ratings

In [30]:
df_ratings.head()

,old_user_id,item_id,timestamp,rating,able,absolutely,acoustic,actually,add,admit,...,write,writing,written,wrong,year,years,yes,young,user_id,item_id
0,A000710818SILS4SMRH43,B00A3OT5A4,2013-07-18,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,903
87,A00167062N853P9T0TEJ5,B004D1GZ2E,2013-02-26,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,767
2040,A00219482AZDX81KAXHEF,B0000029B0,2013-03-09,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,96
2244,A00361823YYOF5UOJPIU,B007B6VOII,2012-11-05,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,825
2495,A00473363TJ8YSZ3YAGG9,B00CBUNUAO,2013-12-07,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,944


In [31]:
print(df_ratings.shape)
print(df_items.shape)

(203370, 506)
(999, 6)


In [32]:
df_ratings.to_csv("NLP_cleaned_ratings.csv", index=False)
df_items.to_csv("NLP_cleaned_items.csv", index=False)